# <h1><center>Исследование надёжности заёмщиков</center></h1>

## Описание данных
- `children` — количество детей в семье
- `days_employed` — общий трудовой стаж в днях
- `dob_years` — возраст клиента в годах
- `education` — уровень образования клиента
- `education_id` — идентификатор уровня образования
- `family_status` — семейное положение
- `family_status_id` — идентификатор семейного положения
- `gender` — пол клиента
- `income_type` — тип занятости
- `debt` — имел ли задолженность по возврату кредитов
- `total_income` — ежемесячный доход
- `purpose` — цель получения кредита

## Импорт и загрузка данных

In [2]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [3]:
df = pd.read_csv('../../datasets/chapter_02_project_bank.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     19351 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         21525 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      19351 non-null  float64
 12  purpose           21525 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


In [5]:
df.describe()

,Unnamed: 0,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,10762.000000,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,6213.876608,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,5381.000000,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,10762.000000,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,16143.000000,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,21524.000000,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
df.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Результат первого знакомства с данными:
- Файл содержит 21 525 строк
- Есть пропуски данных в столбцах: `days_employed`, `total_income`.
- Похоже, что присутствуют "выбросы" в обе стороны. 401 755 дней трудового стажа - это как-то многовато. А -18 388 дней трудового стажа это как-то странно )
- В `education`, `family_status`, `income_type` - содержатся значения в разных регистрах, надо исправлять
- В `purpose` есть все необходимое для улучшения навыков по лемматизации )

## Предобработка данных
Начну с самого простого - приведу все текстовые данные к единому (нижнему) регистру. 

In [7]:
def my_lower(name):
    '''
    Функция принимает имя столбца и приводит все текстовые данные к нижнему регистру
    '''
    df[name] = df[name].str.lower()

In [8]:
my_lower('education')
my_lower('family_status')
my_lower('gender')
my_lower('purpose')

Следующим шагом рассмотрю количество записей в группах `income_type`

In [9]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [10]:
df['income_type'].value_counts()[4:8].sum()/len(df)*100

0.027874564459930314

Так как суммарное количество записей в группах: *предприниматель, безработный, в декрете, студент*  дает всего 2 сотых процента от общего числа, то разрешу себе удалить эти данные, как нерепрезентативные. 

In [11]:
df = df[
    (df['income_type'] != 'предприниматель') &
    (df['income_type'] != 'безработный') &
    (df['income_type'] != 'студент') &
    (df['income_type'] != 'в декрете')
]

In [12]:
df['income_type'].value_counts()

сотрудник      11119
компаньон       5085
пенсионер       3856
госслужащий     1459
Name: income_type, dtype: int64

Теперь проверю значения в столбце `children`

In [13]:
df['children'].value_counts()

 0     14145
 1      4817
 2      2054
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Мои предположения (допущения):
1.	-1 – это ошибка ввода данных. Возможно, что должны были ввести одного ребенка. 
2.	20 детей – тоже ошибка ввода данных. Возможно, что ноль лишний добавился случайно. Считаю так из-за того, что возраст некоторых родителей 20 детей не позволил бы им завести детей в таком количестве (молодые)

In [14]:
# Меняю данные 
df.loc[(df['children'] == -1), 'children'] = 1
df.loc[(df['children'] == 20), 'children'] = 2

In [15]:
# Проверка
df['children'].value_counts()

0    14145
1     4864
2     2130
3      330
4       41
5        9
Name: children, dtype: int64

## Обработка пропусков

Пропуски значений (именнно NaN, а не 0) есть в двух местах: `days_employed` и `total_income`. Считаю, что пропуски нужно заменить на 0, чтобы была возможность быстро проводить операции над данными столбцами. Считаю, что пропуски нужно заменить на 0, чтобы была возможность быстро проводить операции над данными столбцами. Но 0 буду ставить только в том случае, если в данных нет изначальных 0. Иначе потом не смогу отличить «свои» нули от изначальных. 

In [16]:
def my_nan_zero(df, name):
    if len(df[df[name] == 0]) == 0:
        df[name] = df[name].fillna(0)
        result = df[df[name] == 0].count()[name]
        print('В столбце {} было заменено {} строк'.format(name, result) )
    else:
        print('Столбец {} уже содержит 0. Бездушная замена Nan на новые 0 может запутать данные.'.format(name))

In [17]:
my_nan_zero(df, 'days_employed')

В столбце days_employed было заменено 2173 строк


In [18]:
my_nan_zero(df, 'total_income')

В столбце total_income было заменено 2173 строк


Меня очень смутило такое точное совпадение количества замен. Получается, что в исходных данных было 2174 строки с NaNами по `days_employed` и `total_income`. Попробую посмотреть данные строки более внимательно, может быть будет какая-то связь или закономерность. 

In [19]:
# df[(df['total_income'] == 0) & (df['days_employed'] == 0)]
# Рассмотрел данные в ручном режиме (глазми) по различным срезам. 
# Какой-то явной закономерности не увидел. 
# Думаю, что в дальнейшем надо будет заменить days_employed и total_income на средние значения для данных групп.

Надо разобраться с нулевыми и отрицательными значениями в столбце `days_employed`. Для удобства анализа переведу дни в годы.

In [20]:
# Округляю значения для дальнейшего удобства работы с данными
df['years_employed'] = round(df['days_employed'] / 365, 1)

In [21]:
df['years_employed'].describe()

count    21519.000000
mean       155.235211
std        369.463012
min        -50.400000
25%         -6.900000
50%         -2.700000
75%          0.000000
max       1100.700000
Name: years_employed, dtype: float64

Первое, что бросается в глаза - **75% данных сосредоточены ниже 0**. Это отрицательные значения. Предположу, что это ошибка ввода данных (человеческий фактор или сбой программы).

In [22]:
# Убираем отрицательные значения years_employed
df['years_employed'] = df['years_employed'].abs()
df['years_employed'].describe()

count    21519.000000
mean       164.764315
std        365.312834
min          0.000000
25%          1.700000
50%          5.000000
75%         13.100000
max       1100.700000
Name: years_employed, dtype: float64

Я не стал округлять перевод дней в годы до целого числа, так как в таком случае стаж в несколько месяцев был бы равен нулю. А это не совсем корректно и возможно скажется на точности итогового прогноза по возврату кредита. Поэтому оставил округление до десятых. После такого преобразования вылезла следующая проблема - данных получилось много и сделать "короткую" сводную таблицу не смог. Если делать свод по новому столбцу `years_employed`, то получается очень много значений (привет, округления до дестях) и это неудобно анализировать. Наверное как-то можно, но я пока не понимаю как. 

Поэтому сделал следующее. Написал функцию которая считает количество записей в зависимости от введенных значений `years_employed`.
Логика была следующая. Мне надо понять сколько записей:
- с трудовым стажем = 0 
- с трудовым стажем в диапазоне больше 0 и меньше 50 лет 
- с трудовым стажем в диапазоне больше 50 и меньше 100 лет
- с трудовым стажем в диапазоне больше 100 лет



In [23]:
def count_group(name, one, two, three):
    '''
    Функция принимает имя столбца и три числовых значения = границам групп для подсчета количества записей в этих группах
    '''
    group_1 = df[df[name] == one].count()[name]
    group_2 = df[(df[name] > one) & (df[name] <= two)].count()[name]
    group_3 = df[(df[name] > two) & (df[name] <= three)].count()[name]
    group_4 = df[(df[name] > three)].count()[name]
    total = group_1 + group_2 + group_3 + group_4

    return print('Стаж {} лет. Кол-во записей {};\n'
                'Стаж до {} лет. Кол-во записей {};\n'
                 'Стаж до {} лет. Кол-во записей {};\n'
                 'Стаж более {} лет. Кол-во записей {};\n'
                 'Всего записей {}, ничего не потеряли '.format(one, group_1, two, group_2, three, group_3, three, group_4, total))

In [24]:
count_group('years_employed', 0 ,50 ,100)

Стаж 0 лет. Кол-во записей 2173;
Стаж до 50 лет. Кол-во записей 15902;
Стаж до 100 лет. Кол-во записей 1;
Стаж более 100 лет. Кол-во записей 3443;
Всего записей 21519, ничего не потеряли 


Весь смысл такой группировки и подсчета (как мне кажется) – найти группы которые не соответствуют действительности. 
Все, что укладывается в стаж до 50 лет считаю корректным, эти значения оставляю без изменений. В диапазон стада от 50 до 100 лет попадает только одна запись. А вот все, что выше 100 лет – это, скорее всего, очередная ошибка ввода данных. Если предположить, что у данных пользователей стаж указан не в днях, а в часах, то цифры становятся более похожими на правду. 

In [25]:
df[(df['years_employed'] > 50) & (df['years_employed'] < 100)]

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
16335,16335,1,-18388.949901,61,среднее,1,женат / замужем,0,f,сотрудник,0,186178.934089,операции с недвижимостью,50.4


In [26]:
# Пересчитаю аномально большой стаж с учетом часов и сохраню значения в новый столбец `years_to_hours`
df.loc[df['years_employed'] > 100, 'years_to_hours'] = round(df['years_employed'] / 24, 1)

# в новый столбец true_employed соберу корректный стаж из years_employed
df.loc[df['years_employed'] < 100, 'true_employed'] = df['years_employed']

# и из years_to_hours
df.loc[df['years_to_hours'].notnull(), 'true_employed'] = df['years_to_hours']

In [27]:
# Заменяю NaN на нули
my_nan_zero(df, 'years_to_hours')

В столбце years_to_hours было заменено 18076 строк


In [28]:
df['true_employed'].describe()

count    21519.000000
mean        11.431228
std         14.429693
min          0.000000
25%          1.700000
50%          5.000000
75%         13.100000
max         50.400000
Name: true_employed, dtype: float64

Буду считать, что на вопрос **Что делать с аномально большим стажем?** я ответил и преобразовал данные к более-менее реальным. 
Признаюсь честно - есть ощущение, что что-то мог упустить ) 

Теперь попробую разобраться с вопросом **Что делать с нулевым стажем?** Количество записей с нулевым стажем достаточно большое (2 174), поэтому удалять их не вариант. Попробую восстановить стаж от средних значений по группам в `true_employed`

In [29]:
# Соберу уникальные значения income_type для true_employed = 0 в список
array = df[df['true_employed'] == 0]['income_type'].unique()

income_type_list = []
for i in array:
    income_type_list.append(i)
    
income_type_list

['пенсионер', 'госслужащий', 'компаньон', 'сотрудник']

In [30]:
def mean_income_type_group(name):
    '''
    Функция получает название группы ('пенсионер', 'госслужащий' и т.д.) 
    и считает среднее значение стажа для всех представителей данной группы у которых стаж не равен нулю
    '''
    
    mean = df[(df['true_employed'] != 0) & (df['income_type'] == name)]['true_employed'].mean().round()
    df.loc[(df['true_employed'] == 0) & (df['income_type'] == name), 'true_employed'] = mean

In [31]:
mean_income_type_group('пенсионер')
mean_income_type_group('госслужащий')
mean_income_type_group('компаньон')
mean_income_type_group('сотрудник')

AttributeError: 'float' object has no attribute 'round'

In [33]:
len(df[df['income_type'] == 0])

0

Надо разобраться с нулевыми значениями в столбце `total_income`. Таких пропусков 2 173. Попробую заменить доход средними или медианными значениями по группам из `income_type`

Первым делом надо проверить данные на выбросы.

In [34]:
# Основные статистики для всех у кого доход не равен нулю 
# Округление поставил для удобства восприятия цифр
df[df['total_income'] != 0]['total_income'].describe().round()

count      19346.0
mean      167418.0
std       102950.0
min        20667.0
25%       103085.0
50%       145021.0
75%       203440.0
max      2265604.0
Name: total_income, dtype: float64

Максимум подозрения вызывают зарплаты больше 2 млн в месяц. Надо посмотреть данные более внимательно

In [35]:
df[df['total_income'] > 2000000]

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,years_to_hours,true_employed
12412,12412,0,-1477.438114,44,высшее,0,женат / замужем,0,m,компаньон,0,2.265604e+06,ремонт жилью,4.0,0.0,4.0
19606,19606,1,-2577.664662,39,высшее,0,женат / замужем,0,m,компаньон,1,2.200852e+06,строительство недвижимости,7.1,0.0,7.1


Всего две записи с такими большими доходами. Я не сомневаюсь, что  у «компаньонов» возможны такие доходы. Меня смущает другое:
1.	Из 5 085 компаньонов в df всего дав имеют такой высокий доход 
2.	Один из них имеет просрочку по ежемесячным выплатам. Это странно. 

Сделаю допущение, что эти две записи – это ошибка ввода данных. Заменю столь высокий доход на медиану. Даже если в данных нет ошибки, то моя манипуляция не должна сильно испортить обую картину. Ведь это всего 2 записи из 5 085 «компаньонов».

In [36]:
# Меняю выбросы на медиану
df.loc[df['total_income'] > 2000000, 'total_income'] = df[df['total_income'] != 0]['total_income'].describe()[5]

Теперь произведу замену нулей на значения медианы.Делаю функцию которая получает на вход элементы списка из п1 и делает следующее: находит всех в указанной категории чей доход равен нулю. Меняет его нулевой доход на медианное значение для данной категории 


In [37]:
# Получаю список групп которые имеют доход равный нулю
array_text = df[df['total_income'] == 0]['income_type'].unique()

my_list = []
for i in array_text:
    my_list.append(i)

In [38]:
def median_total_income(name):
    # Получаю медиану для заданной группы и среди тех, чей доход НЕ равен нулю
    median = df[(df['total_income'] != 0) & (df['income_type'] == name)]['total_income'].median()
    
    # Меняю доход равный нулю на медиану для заданной группы
    df.loc[(df['total_income'] == 0) & (df['income_type'] == name), 'total_income'] = median

In [39]:
# Применяю функцию в цикле к списку
for i in my_list:
    median_total_income(i)

In [40]:
# Проверяю, что нулей не осталось 
len(df[df['total_income'] == 0])

0

In [41]:
# Удаляю строчку с полом xna
df = df.loc[df['gender'] != 'xna']

Данный блок работы занял у меня больше всего времени (застрял на обработке стажа).
Основной вывод - внимательно читать задачу и рассуждать (размышлять) о том, что не обходимо для ее выполнения.

## Замена типа данных

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21518 entries, 0 to 21524
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21518 non-null  int64  
 1   children          21518 non-null  int64  
 2   days_employed     21518 non-null  float64
 3   dob_years         21518 non-null  int64  
 4   education         21518 non-null  object 
 5   education_id      21518 non-null  int64  
 6   family_status     21518 non-null  object 
 7   family_status_id  21518 non-null  int64  
 8   gender            21518 non-null  object 
 9   income_type       21518 non-null  object 
 10  debt              21518 non-null  int64  
 11  total_income      21518 non-null  float64
 12  purpose           21518 non-null  object 
 13  years_employed    21518 non-null  float64
 14  years_to_hours    21518 non-null  float64
 15  true_employed     21518 non-null  float64
dtypes: float64(5), int64(6), object(5)
memor

In [54]:
df['days_employed'] = df['days_employed'].astype('int64')
df['total_income'] = df['total_income'].astype('int64')
df['years_employed'] = df['years_employed'].astype('int64')
df['years_to_hours'] = df['years_to_hours'].astype('int64')
df['true_employed'] = df['true_employed'].astype('int64')

## Лемматизация

In [55]:
m = Mystem()

In [50]:
# Собираю уникальные значения из purpose и перевожу их в строку
array_text = df['purpose'].unique()

text = []
for i in array_text:
    text.append(i)

text = ' '.join(text)

In [51]:
result = Counter(m.lemmatize(text))
print(result)

Counter({' ': 96, 'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'подержать': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'со': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1, '\n': 1})


In [52]:
# Заполняю df категориями по целям выданного кредита
# Признаюсь честно, функцию подглядел в треде. Сам не сообразил реализовать подобную красоту 

keywords = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба', 'ремонт']
 
def add_keyword(str):
    lemmed = m.lemmatize(str)
    for keyword in keywords:
        if keyword in lemmed:
            return keyword
 
df['purp_keyword'] = df['purpose'].apply(add_keyword) 

In [53]:
df.head()

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,years_to_hours,true_employed,purp_keyword
0,0,1,-8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,23,0,23,жилье
1,1,1,-4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,11,0,11,автомобиль
2,2,0,-5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,15,0,15,жилье
3,3,3,-4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,11,0,11,образование
4,4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,932,38,38,свадьба


Все разнообразие целей получения кредита (было 38 уникальных описаний) удалось свести к шести основным темам. И это круто. Так как проанализировать шесть категорий сильно проще чем 38 )

## Категоризация данных
Буду делать следующие категории:
- debt: есть просрочка / нет просрочки
- дети: есть / нет. 
- семейное положение. 
- уровень дохода
- по целям кредита

In [56]:
# debt
debt_0 = df[df['debt'] == 0].count()[0]
debt_1 = df[df['debt'] == 1].count()[0]

In [57]:
# children
# Добавляю новый столбец для формирования статуса есть дети или нет
# 1 тем у кого дети есть
# 0 тем у кого детей нет
df.loc[(df['children'] != 0), 'children_status'] = 1

# Заменяю Nan на нули с помощью ранее написанной функции
my_nan_zero(df, 'children_status')

# Меняю тип данных у нового столбца
df['children_status'] = df['children_status'].astype('int64')

# Сводная таблица по детям и выплате кредита
group_children = df.pivot_table(index='debt', columns='children_status', aggfunc={'children_status':'count'})

В столбце children_status было заменено 14144 строк


In [58]:
# family_status
# Сводная таблица по семейному положению и выплате кредита
group_family = df.pivot_table(index='debt', columns='family_status', aggfunc={'family_status':'count'})
group_family

family_status                                                  \
family_status     в разводе вдовец / вдова гражданский брак женат / замужем   
debt                                                                          
0                      1110            897             3786           11448   
1                        85             63              388             929   

                                     
family_status не женат / не замужем  
debt                                 
0                              2538  
1                               274

In [59]:
# Сделаю предположение, что проще будет свести типы семейного положения к двум состояниям: есть пара, нет пары
def family_status_new(income):
    if income == 'гражданский брак' or income == 'женат / замужем':
        return 1
    else: 
        return 0
    
    
# Запускаю функцию
df['family_status_new'] = df['family_status'].apply(family_status_new)

# Делаю новую группировку
group_family = df.pivot_table(index='debt', columns='family_status_new', aggfunc={'family_status':'count'})
group_family

family_status       
family_status_new             0      1
debt                                  
0                          4545  15234
1                           422   1317

In [60]:
# total_income
# Как и в прошлый раз я не вижу других вариантов разбивки дохода кроме как по квантилям. 
# Поэтому повторю эту процедуру и сгруппирую доход по ним. 

# Получаю значения квантилий по доходу 
my_quantile = df['total_income'].quantile([0.25, 0.5, 0.75])

# Функция присваивает статус дохода в зависимости от цифры дохода 
def total_income_group(income):
  
    if income <= my_quantile[0.25]:
        return 'min'
    elif income > my_quantile[0.25] and income <= my_quantile[0.5]:
        return 'low_middle'
    elif income > my_quantile[0.5] and income <= my_quantile[0.75]:
        return 'hi_middle'
    else:
        return 'max'

# Запуская функцию 
df['group_income'] = df['total_income'].apply(total_income_group) 

# Делаю новую группировку
group_income = df.pivot_table(index='debt', columns='group_income', aggfunc={'group_income':'count'})

In [61]:
# purp_keyword

# Делаю  группировку
group_purpose = df.pivot_table(index='debt', columns='purp_keyword', aggfunc={'purp_keyword':'count'})

Главный вывод данного раздела - стараться группировать данные так, чтобы потом было удобно отвечать на поставленные вопросы. 

### Поиск зависимости между наличием детей и возвратом кредита

In [62]:
group_children = df.pivot_table(index='debt', columns='children_status', values='children', aggfunc='count', margins=True)
group_children

children_status,0,1,All
debt,,,
0,13081,6698,19779
1,1063,676,1739
All,14144,7374,21518


In [63]:
# Теперь добавлю долю должников 
group_children['share_children_no'] = round((group_children[0] / group_children['All']) * 100)
group_children['share_children_yes'] = round((group_children[1] / group_children['All']) * 100)
# group_children

group_children = group_children[['share_children_no', 'share_children_yes']]
group_children

children_status,share_children_no,share_children_yes
debt,,
0,66.0,34.0
1,61.0,39.0
All,66.0,34.0


Сводную таблицу читаю «горизонтально». Делаю это для того чтобы попытаться ответить на вопрос - какой процент должников с детьми/без детей среди всех должников. 
Получается, что среди всех должников:
- 39% это должники с детьми
- 61% это должники без детей. 
По этим данным делаю вывод, что наличие детей влияет на возврат кредита в срок. Люди с детьми стараются не нарушать условия кредитного договора и платят в установленные временные рамки.

### Поиск зависимости между семейным положением и возвратом кредита в срок?

In [64]:
group_family = df.pivot_table(index='debt', columns='family_status_new', values='children', aggfunc='count', margins=True)
group_family

family_status_new,0,1,All
debt,,,
0,4545,15234,19779
1,422,1317,1739
All,4967,16551,21518


In [65]:
# Теперь добавлю долю должников 
group_family['share_family_no'] = round((group_family[0] / group_family['All']) * 100)
group_family['share_family_yes'] = round((group_family[1] / group_family['All']) * 100)
# group_family

# Оставляю только столбцы с долями
table = group_family[['share_family_no', 'share_family_yes']]
table

family_status_new,share_family_no,share_family_yes
debt,,
0,23.0,77.0
1,24.0,76.0
All,23.0,77.0


Сводную таблицу читаю «горизонтально». Делаю это для того чтобы попытаться ответить на вопрос - какой процент должников с парой/без пары среди всех должников. 
Получается, что среди всех должников:
- 24% это должники без пары
- 76% это должники с парой. 
По этим данным делаю вывод, что просрочка платежа встречается чаще у «семейных» заемщиков. Люди живущее без пары чаще выплачивают кредит в срок. 

### Поиск зависимости между уровнем дохода и возвратом кредита в срок?

In [66]:
group_income = df.pivot_table(index='debt', columns='group_income', values='children', aggfunc='count', margins=True)
group_income

group_income,hi_middle,low_middle,max,min,All
debt,,,,,
0,4790,5038,4995,4956,19779
1,446,483,385,425,1739
All,5236,5521,5380,5381,21518


In [67]:
# Теперь добавлю долю должников 
group_income['share_hi_middle'] = round((group_income['hi_middle'] / group_income['All']) * 100)
group_income['share_low_middle'] = round((group_income['low_middle'] / group_income['All']) * 100)
group_income['share_max'] = round((group_income['max'] / group_income['All']) * 100)
group_income['share_min'] = round((group_income['min'] / group_income['All']) * 100)
# group_income

# Оставляю только столбцы с долями
table = group_income[['share_hi_middle', 'share_low_middle', 'share_max', 'share_min']]
table

group_income,share_hi_middle,share_low_middle,share_max,share_min
debt,,,,
0,24.0,25.0,25.0,25.0
1,26.0,28.0,22.0,24.0
All,24.0,26.0,25.0,25.0


Мне трудно оценить корректно ли говорить, что разница в 6% (28% максимальное значение и 22% минимальное значение) между группами говорит о том, что группа хуже платит. Но буду предполагать, что да. Поэтому вывод такой:
1. Максимальная доля должников находится в группе `low_middle` (ниже среднего дохода). Это 28% заемщиков. 
2. Минимальная доля должников находится в группе `max`. Что ожидаемо, но удивительно. Видимо не только уровень дохода виляет на выплату в срок. Возможно в этой группе есть люди с парой (предыдущий вопрос)

Итоговый ответ на вопрос - да, доход влияет на выплату кредита в срок.

### Поиск зависимости между целью кредита и его возврат вом срок?

In [68]:
group_purpose = df.pivot_table(index='debt', columns='purp_keyword', values='children', aggfunc='count', margins=True)
group_purpose

purp_keyword,автомобиль,жилье,недвижимость,образование,свадьба,All
debt,,,,,,
0,3912,4164,5890,3652,2161,19779
1,402,307,474,370,186,1739
All,4314,4471,6364,4022,2347,21518


In [69]:
group_purpose['share_автомобиль'] = round((group_purpose['автомобиль'] / group_income['All']) * 100)
group_purpose['share_жилье'] = round((group_purpose['жилье'] / group_income['All']) * 100)
group_purpose['share_недвижимость'] = round((group_purpose['недвижимость'] / group_income['All']) * 100)
group_purpose['share_образование'] = round((group_purpose['образование'] / group_income['All']) * 100)
group_purpose['share_свадьба'] = round((group_purpose['свадьба'] / group_income['All']) * 100)
# group_purpose

# Оставляю только столбцы с долями
table = group_purpose[['share_автомобиль', 'share_жилье', 'share_недвижимость', 'share_образование', 'share_свадьба']]
table

purp_keyword,share_автомобиль,share_жилье,share_недвижимость,share_образование,share_свадьба
debt,,,,,
0,20.0,21.0,30.0,18.0,11.0
1,23.0,18.0,27.0,21.0,11.0
All,20.0,21.0,30.0,19.0,11.0


Отвечая на поставленый вопрос можно с уверенностью сказать, что цель взятия кредита влияет на выплату в срок. Максимальное количество должников - это группа недвижимость (покупка жилья). 
А вот молодожены хоть и не всегда платят в срок, но они самые пунктуальные.

## Итоговый вывод
Основные выводы по проделанной работе:
- Представленный набор данных содержал пропуски значений в `days_employed` и `total_income`. Большую часть времени потратил на обработку пропусков `days_employed`. Как выяснилось позже - зря. Ответы на финальные вопросы не потребовали работы с переменной `days_employed`.  А вот пропуски в `total_income` были критичны для решения поставленной задачи. Но их было меньше и обрабатывались они быстрее. Так же встречались и ошибки ввода данных. Например количество детей равное -1 или трудовой стаж равный 1100 лет. 

- Переменные `education`, `family_status`, `income_type` содержали дубли в различных регистрах. Это все пришлось исправлять. 

- Переменная `income_type` содержала нерепрезентативные записи по группам предприниматель, безработный, в декрете, студент. Общее количество таких записей было на уровне 0.03%. Такое количество информации не позволяет делать корректные выводы. Данные были удалены. 

- Для выявления основных причин взятия кредита была обработана переменная `purpose`. Было найдено 38 уникальных словосочетаний обозначающих цель получения кредита. Данные словосочетания были обработаны и сгруппированы в шесть основных групп.

- Для дальнейшего анализа данных и ответов на поставленные вопросы были сформированы следующие группировки данных:
    - Выплата кредита: в срок или нет
    - Дети: есть или нет
    - Семейное положение
    - Доход
    - Цель получения кредита 

- Результат исследования показал, что
    - Заемщики с детьми выплачивают кредит лучше, чем заемщики без детей. Возможно, что это вызвано необходимостью думать о будущем и планировать доходы/расходы семьи. 
    - При этом, при анализе влияния семейного положения на выплату кредита в срок получилось, что люди «с парой» (замужем/женат/гражданский брак) чаще допускают срок выплаты кредита. Было бы полезно (и интересно) проанализировать пересечение таких семей с наличием детей. Сейчас не успеваю это сделать, но задачку себе поставил ) 
    - Если сравнивать выплату кредита и уровень дохода, то максимальное количество должников - это заемщики с уровнем дохода «нижняя середина» (больше первого квартиля, но меньше медианы). Странно, что достаточно большая часть должников (22%) есть и среди заемщиков с максимальным уровнем дохода. Было бы интересно проанализировать их пересечение с целями получения кредита, детьми, семейным положением. 
    - Чаще всего срок выплаты нарушают заемщики с целью получения кредита на покупку недвижимости. Возможно это связано с большими суммами кредита (скорее всего это ипотека) и различными жизненными обстоятельствами. 